# ALL_RSS

## Given that RSS FEEDs update frequently we need to find a way to include the new jobs into the current DB without losing the "old" jobs.

### In brief...

### For every iteration, the df is inspected and then filtered by its pubDate (if there's one). Finally, it creates a new df that will be transformed into a new table in the local postgreSQL DB.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import mysql.connector
import psycopg2
import numpy as np

In [2]:
df = pd.read_csv('/Users/juanreyesgarcia/Library/CloudStorage/OneDrive-FundacionUniversidaddelasAmericasPuebla/DEVELOPER/PROJECTS/CRAWLER_ALL/yummy_soup_rss.csv')

##### Play with the settings...

In [3]:
pd.set_option('display.max_colwidth', 150)

In [4]:
pd.set_option("display.max_rows", None)

# EDA Checklist 

In [5]:
df.head()

,title,link,description,pubDate,location
0,Senior Clojure Developer Codete,https://ClojureJobboard.com/clojure-job/ib-remote-senior-clojure-developer-krakow-codete-remotework.html,Remote Codete is not just a software company its a place where techenthusiasts can grow by doing what they love and feel valued for what they are...,NaN,NaN
1,Senior Software Engineer CIS Cloud Services Centripetal,https://ClojureJobboard.com/clojure-job/ha-remote-senior-software-engineer-cis-cloud-services-centripetal-remotework.html,Remote Do you want to be at the forefront of intelligencedriven cybersecurity We at Centripetal are innovators of disruptive cybersecurity soluti...,NaN,NaN
2,FullStack Clojure Developer Appear,https://ClojureJobboard.com/clojure-job/ia-remote-europe-full-stack-clojure-developer-appear-remotework.html,Remote At Appear were looking for a FullStack Clojure Developer to join our team Fulltime Oslo OR Remote About the role Were looking for a Cloju...,NaN,NaN
3,Software Engineer Clojure Backend Comcast,https://ClojureJobboard.com/clojure-job/ic-software-engineer-clojure-backend-philadelphia-comcast.html,Philadelphia United States Comcasts Technology Product Experience organization works at the intersection of media and technology Our innovative ...,NaN,NaN
4,Senior Application Developer Clojure Pennymac,https://ClojureJobboard.com/clojure-job/ii-senior-application-developer-clojure-agoura-hills-pennymac.html,Agoura Hills United States At Pennymac our people are the foundation of our success and at the heart of our dynamic work culture Together we work...,NaN,NaN


In [6]:
df.tail()

,title,link,description,pubDate,location
1389,ReactJS Lead,https://jobs.wordpress.net/job/reactjs-lead/,WordPress WooCommerce ReactJS React Native NodeJS Axio KDC is moving all its custom themes to Reactjs to get a better UX for its end clients Also ...,Thu 02 Mar 2023 131719 0000,NaN
1390,WordPress developer,https://jobs.wordpress.net/job/wordpress-developer-792/,Respacio provides services to the international real estate industry We provide both websites and plugins for clients which synchronise with the R...,Thu 02 Mar 2023 131657 0000,NaN
1391,3 yrs WP Developer Sites Custom Themes Nice To Have UIUX skills,https://jobs.wordpress.net/job/3-yrs-wp-developer-sites-custom-themes-nice-to-have-ui-ux-skills/,Our global team is looking for a long term relationship with a US based WordPress developer You must be a legal resident or US citizen to be consi...,Thu 02 Mar 2023 131646 0000,NaN
1392,WordPress Site Builder,https://jobs.wordpress.net/job/wordpress-site-builder/,Luma Sites is a design company focused on designing building hosting and maintaining WordPress websites for our clients With over a decade of expe...,Thu 02 Mar 2023 131621 0000,NaN
1393,Front End Developer,https://jobs.wordpress.net/job/front-end-developer-47/,We are looking for a wellrounded web developer to provide design 38 programming support on a contract basis which could turn into a longterm relat...,Tue 28 Feb 2023 130702 0000,NaN


### Describe the DF (current)

In [7]:
df.describe()

,title,link,description,pubDate,location
count,1394,1394,1159,1217,645
unique,1273,1394,1128,1156,180
top,DevOps Engineer,https://ClojureJobboard.com/clojure-job/ib-remote-senior-clojure-developer-krakow-codete-remotework.html,United States,20230225T2311040000,Remote
freq,7,1,11,7,88


### Checking for missing values DF (current)

In [8]:
df.isnull().sum()

title            0
link             0
description    235
pubDate        177
location       749
dtype: int64

### Fill missing values with "NaN" DF (current)

In [9]:
# Fill missing values with "NaN"
df.fillna("NaN", inplace=True)

In [10]:
df.isnull().sum()

title          0
link           0
description    0
pubDate        0
location       0
dtype: int64

### Checking for duplicates DF (current)

In [11]:
df.duplicated().sum()

0

In [12]:
df.describe()

,title,link,description,pubDate,location
count,1394,1394,1394,1394,1394
unique,1273,1394,1129,1157,181
top,DevOps Engineer,https://ClojureJobboard.com/clojure-job/ib-remote-senior-clojure-developer-krakow-codete-remotework.html,NaN,NaN,NaN
freq,7,1,235,177,749


In [53]:
df.dtypes

title          object
link           object
description    object
pubDate        object
location       object
dtype: object

# From string to date time...

In [13]:
df.loc[:, 'pubDate']  = pd.to_datetime(df['pubDate'], errors="coerce", format="%a %d %b %Y", exact=False)

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_80120/2269829544.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'pubDate']  = pd.to_datetime(df['pubDate'], errors="coerce", format="%a %d %b %Y", exact=False)


## Filter rows by a date range (this reduces the number of rows... duh)

In [14]:
# filter rows by a date range
start_date = pd.to_datetime('2016-01-01')
end_date = pd.to_datetime('2023-02-15')

#for df
date_range_filter = (df['pubDate'] >= start_date) & (df['pubDate'] <= end_date)
df = df.loc[~date_range_filter]

In [15]:
df = df.sort_values(by='pubDate')

In [16]:
df.describe(datetime_is_numeric=True)

,pubDate
count,386
mean,2023-02-25 18:27:58.756476416
min,2023-02-16 00:00:00
25%,2023-02-22 00:00:00
50%,2023-02-28 00:00:00
75%,2023-03-02 00:00:00
max,2023-03-03 00:00:00


In [17]:
df

,title,link,description,pubDate,location
63,Principal software engineer SQL API at Cube Dev San Francisco CA USA,https://www.crunchboard.com/jobs/197021714-principal-software-engineer-sql-api-at-cube-dev,At Cube Dev we are building a technology stack for modern analytics If you are fascinated by the software that powers large technology companies but,2023-02-16,NaN
882,Drupal FrontEnd Engineer Remote US amp CANADA position is open kanopistudios,https://jobs.drupal.org/kanopi-studios/job/17548,United States,2023-02-16,NaN
62,Senior Full Stack Developer NET at AAOS Rosemont IL USA,https://www.crunchboard.com/jobs/197057956-senior-full-stack-developer-net-at-aaos,You are an enthusiastic thought leader who coaches and mentors those around you You are highly creative and understand how to balance creativity and,2023-02-16,NaN
1257,Full Stack Developer,https://vuejobs.com/jobs/my-worksheet-maker-full-stack-developer?utm_source=rssfeed,Employer My Worksheet Maker Location Full Stack Developer FullTime RemotenbspAre you a skilled developer looking to use your technical expertise ...,2023-02-16,NaN
788,Riva Health is hiring for a position of Lead Product Designer Riva Health in United States Canada,https://dribbble.com/jobs/203123-Lead-Product-Designer-Riva-Health?source=feed,NaN,2023-02-16,NaN
792,Johns Hopkins University is hiring for a position of Web Designer in Remote,https://dribbble.com/jobs/203090-Web-Designer?source=feed,NaN,2023-02-16,NaN
1256,UX and Design Focused Senior Frontend Developer US Only,https://vuejobs.com/jobs/drive-commerce-ux-and-design-focused-senior-front-end-developer?utm_source=rssfeed,Employer Drive Commerce Location United States We are building the nextgeneration ecommerce personalization platform and we are looking for you if...,2023-02-16,NaN
1353,Aha Senior Product Marketing Manager,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Headquarters United States URL httpwwwahaio Aha helps companies build lovable software We provide the worlds 1 product development tools Aha Ro...,2023-02-16,NaN
793,Voormedia is hiring for a position of Product Designer in Amsterdam Netherlands,https://dribbble.com/jobs/203058-Product-Designer?source=feed,NaN,2023-02-16,NaN
785,Revelxyz is hiring for a position of UI UX Designer in Remote,https://dribbble.com/jobs/203189-UI-UX-Designer?source=feed,NaN,2023-02-17,NaN


In [18]:
df.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_80120/3627053830.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,title,link,description,pubDate,location
count,631,631,631,386,631
unique,571,631,504,16,69
top,Senior Software Engineer,https://www.crunchboard.com/jobs/197021714-principal-software-engineer-sql-api-at-cube-dev,NaN,2023-03-03 00:00:00,NaN
freq,6,1,108,71,475
first,NaN,NaN,NaN,2023-02-16 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-03 00:00:00,NaN


In [19]:
df.dtypes

title                  object
link                   object
description            object
pubDate        datetime64[ns]
location               object
dtype: object

In [20]:
df['title'] = df['title'].astype(str)
df['link'] = df['link'].astype(str)
df['description'] = df['description'].astype(str)
df['location'] = df['location'].astype(str)

In [21]:
df.dtypes

title                  object
link                   object
description            object
pubDate        datetime64[ns]
location               object
dtype: object

###### Remember 'object' is the pandas data type that is used to represent string data.

In [22]:
df1 = df.copy()

In [23]:
df['description'] = df['description'].str.slice(0, 1000)

In [24]:
# replace NaT values in the DataFrame with None
df = df.replace({np.nan: None, pd.NaT: None})

In [25]:
df

,title,link,description,pubDate,location
63,Principal software engineer SQL API at Cube Dev San Francisco CA USA,https://www.crunchboard.com/jobs/197021714-principal-software-engineer-sql-api-at-cube-dev,At Cube Dev we are building a technology stack for modern analytics If you are fascinated by the software that powers large technology companies but,2023-02-16 00:00:00,NaN
882,Drupal FrontEnd Engineer Remote US amp CANADA position is open kanopistudios,https://jobs.drupal.org/kanopi-studios/job/17548,United States,2023-02-16 00:00:00,NaN
62,Senior Full Stack Developer NET at AAOS Rosemont IL USA,https://www.crunchboard.com/jobs/197057956-senior-full-stack-developer-net-at-aaos,You are an enthusiastic thought leader who coaches and mentors those around you You are highly creative and understand how to balance creativity and,2023-02-16 00:00:00,NaN
1257,Full Stack Developer,https://vuejobs.com/jobs/my-worksheet-maker-full-stack-developer?utm_source=rssfeed,Employer My Worksheet Maker Location Full Stack Developer FullTime RemotenbspAre you a skilled developer looking to use your technical expertise ...,2023-02-16 00:00:00,NaN
788,Riva Health is hiring for a position of Lead Product Designer Riva Health in United States Canada,https://dribbble.com/jobs/203123-Lead-Product-Designer-Riva-Health?source=feed,NaN,2023-02-16 00:00:00,NaN
792,Johns Hopkins University is hiring for a position of Web Designer in Remote,https://dribbble.com/jobs/203090-Web-Designer?source=feed,NaN,2023-02-16 00:00:00,NaN
1256,UX and Design Focused Senior Frontend Developer US Only,https://vuejobs.com/jobs/drive-commerce-ux-and-design-focused-senior-front-end-developer?utm_source=rssfeed,Employer Drive Commerce Location United States We are building the nextgeneration ecommerce personalization platform and we are looking for you if...,2023-02-16 00:00:00,NaN
1353,Aha Senior Product Marketing Manager,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Headquarters United States URL httpwwwahaio Aha helps companies build lovable software We provide the worlds 1 product development tools Aha Ro...,2023-02-16 00:00:00,NaN
793,Voormedia is hiring for a position of Product Designer in Amsterdam Netherlands,https://dribbble.com/jobs/203058-Product-Designer?source=feed,NaN,2023-02-16 00:00:00,NaN
785,Revelxyz is hiring for a position of UI UX Designer in Remote,https://dribbble.com/jobs/203189-UI-UX-Designer?source=feed,NaN,2023-02-17 00:00:00,NaN


# PostgreSQL

### This code creates a new table per iteration

In [26]:
# create a connection to the PostgreSQL database
cnx = psycopg2.connect(user='postgres', password='3312', host='localhost', port='5432', database='postgres')

# create a cursor object
cursor = cnx.cursor()

# get the name of the next table to create
get_table_name_query = '''
    SELECT COUNT(*) FROM information_schema.tables
    WHERE table_name LIKE 'mytable_%'
'''
cursor.execute(get_table_name_query)
result = cursor.fetchone()
next_table_number = result[0] + 1
next_table_name = 'mytable_{}'.format(next_table_number)

# prepare the SQL query to create a new table
create_table_query = '''
    CREATE TABLE {} (
        title VARCHAR(255),
        link VARCHAR(255),
        description VARCHAR(1000),
        pubDate TIMESTAMP,
        location VARCHAR(255)
    )
'''.format(next_table_name)

# execute the create table query
cursor.execute(create_table_query)

# insert the DataFrame into the PostgreSQL database as a new table
for index, row in df.iterrows():
    insert_query = '''
        INSERT INTO {} (title, link, description, pubDate, location)
        VALUES (%s, %s, %s, %s, %s)
    '''.format(next_table_name)
    values = (row['title'], row['link'], row['description'], row['pubDate'], row['location'])
    cursor.execute(insert_query, values)

# commit the changes to the database
cnx.commit()

# close the cursor and connection
cursor.close()
cnx.close()

# MERGE 

In [27]:
#merged = pd.merge(df, df_a, on=['title','link','description','pubDate','location'], how='outer')

In [28]:
#merged.describe()

#### Check for duplicatesss...

In [29]:
#merged.duplicated(subset='link').sum()

#### Drop them...

In [30]:
#df1 = merged.drop_duplicates(subset=['link'])

In [31]:
#df1.duplicated(subset='link').sum()

In [32]:
#df1.describe()

# Save a copy of the df back into yummy_soup_rss_copy.csv so in the next iterations you will merge the output of the crawler with this df (basically making a loop)

## By doing this we will be able to merge the df that comes from the crawler and the merged df. If this was not done (& we just call the yummy_soup_rss_copy.csv file we would only merge those rows with the new ones, without including the data that was already merged)

In [33]:
#df1_copy = df1.copy()
#df1_copy.to_csv('yummy_soup_rss_copy.csv', index=False)

### Explore categorical data

For categorical data, check the number of unique values in each column 

In [34]:
#df1.nunique()

In [35]:
# Visualise the distribution of a categorical column using a bar chart
#df1['pubDate'].value_counts().plot(kind='bar', figsize=(14, 8))
#plt.show()

### Sort the dataframe by the date column in ascending order

In [36]:
#df1 = df1.sort_values(by='pubDate', ascending=False)

### Reset index...

In [37]:
#df1.reset_index(drop=True)

# GIVE ME THOSE JOBSSSS

## Only WorldWide - 2023

#### Given that some dates are NaN it is impossible to create a perfectly tailored df of solely worldwide jobs

##### I have tried to narrow down the search but there is still some jobs that are not fully worldwide. Although, the majority is indeed from anywhere in the world.

In [38]:
key_words = ['worldwide', 'international', 'anywhere']
regex = '|'.join(key_words)


worldwide = df1[df1['title'].str.contains(regex, case=False) | df1['description'].str.contains(regex, case=False) | df1['location'].str.contains(regex, case=False)]

In [39]:
worldwide.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_80120/3825536958.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  worldwide.describe()


,title,link,description,pubDate,location
count,135,135,135,77,135
unique,132,135,134,15,22
top,DevOps Engineer,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Remote Gcore in the nutshell We are an international leader in public cloud and edge computing content delivery hosting and security solutions We...,2023-03-02 00:00:00,NaN
freq,2,1,2,15,67
first,NaN,NaN,NaN,2023-02-16 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-03 00:00:00,NaN


In [40]:
worldwide

,title,link,description,pubDate,location
1353,Aha Senior Product Marketing Manager,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Headquarters United States URL httpwwwahaio Aha helps companies build lovable software We provide the worlds 1 product development tools Aha Ro...,2023-02-16,NaN
313,Junior AffiliateInfluencer Marketing Binance Accelerator Programme 6 months,https://cryptojobslist.com/jobs/junior-affiliate-influencer-marketing-binance-accelerator-programme-6-months-binance-global,Tags Cryptocurrency Jobs Web3 Marketing Jobs Web3 Trading Jobs Web3 Exchange Jobs Blockchain Web3 Jobs Blockchain Junior Level Jobs Blockch...,2023-02-17,Global
312,UIUX Design Intern UED,https://cryptojobslist.com/jobs/ui-ux-design-intern-ued-binance-asia,Tags Cryptocurrency Jobs Cryptocurrency Trading Jobs Blockchain Internship Jobs Blockchain Engineering Jobs Web3 Ui Ux Jobs Blockchain Excha...,2023-02-17,Asia
307,Regulatory Strategy Counsel Europe,https://cryptojobslist.com/jobs/regulatory-strategy-counsel-europe-kraken-digital-asset-exchange-amsterdam,Tags Cryptocurrency Jobs Blockchain Remote Jobs Web3 Non Tech Jobs Web3 Legal Jobs Blockchain Exchange Jobs Cryptocurrency Compliance Jobs ...,2023-02-17,Amsterdam
316,Sr DevOps Engineer New Zealand Fiji GMT12,https://cryptojobslist.com/jobs/sr-devops-engineer-new-zealand-fiji-gmt-12-polygon-auckland,Tags Cryptocurrency Jobs Web3 Remote Jobs Web3 Devops Jobs Blockchain Security Jobs Web3 Developer Jobs Blockchain Engineering Jobs Blockch...,2023-02-17,Auckland
1352,EngagedMD DevOps Engineer,https://weworkremotely.com/remote-jobs/engagedmd-devops-engineer,Headquarters Washington DC URL httpsengagedmdcom Youre an experienced AWS dev ops engineer who is excited to make your mark on our growing engin...,2023-02-17,NaN
322,Senior Software Engineer,https://cryptojobslist.com/jobs/senior-software-engineer-coinbase-remote-india,Tags Cryptocurrency Jobs Blockchain Remote Jobs Web3 Non Tech Jobs Web3 Trading Jobs Cryptocurrency Legal Jobs Cryptocurrency Developer Jobs...,2023-02-17,Remote India
318,Software Engineer,https://cryptojobslist.com/jobs/software-engineer-coinbase-remote-india,Tags Cryptocurrency Jobs Web3 Remote Jobs Blockchain Non Tech Jobs Web3 Trading Jobs Web3 Legal Jobs Web3 Security Jobs Blockchain Develope...,2023-02-17,Remote India
317,Accounts Payable Analyst,https://cryptojobslist.com/jobs/accounts-payable-analyst-parity-technologies-london-england-united-kingdom,Tags Cryptocurrency Jobs Web3 Analyst Jobs Blockchain Operations Jobs Blockchain Finance Jobs Cryptocurrency Accounting Jobs Cryptocurrency ...,2023-02-17,London England United Kingdom
288,HR Payroll Manager FTCMaternity Cover,https://cryptojobslist.com/jobs/hr-payroll-manager-ftc-maternity-cover-gsr-mayfair-greater-london,Tags Cryptocurrency Jobs Web3 Trading Jobs Blockchain Finance Jobs Cryptocurrency Manager Jobs About usFounded in 2013 GSR is a crypto market ...,2023-02-18,Mayfair Greater London


## Analyst - WorldWide

In [41]:

analyst = worldwide[worldwide['title'].str.contains('analyst', case=False) | worldwide['description'].str.contains('analyst', case=False)]


In [42]:
analyst.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_80120/1449045089.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  analyst.describe()


,title,link,description,pubDate,location
count,4,4,4,2,4
unique,4,4,4,2,4
top,Accounts Payable Analyst,https://cryptojobslist.com/jobs/accounts-payable-analyst-parity-technologies-london-england-united-kingdom,Tags Cryptocurrency Jobs Web3 Analyst Jobs Blockchain Operations Jobs Blockchain Finance Jobs Cryptocurrency Accounting Jobs Cryptocurrency ...,2023-02-17 00:00:00,London England United Kingdom
freq,1,1,1,1,1
first,NaN,NaN,NaN,2023-02-17 00:00:00,NaN
last,NaN,NaN,NaN,2023-02-23 00:00:00,NaN


In [43]:
analyst

,title,link,description,pubDate,location
317,Accounts Payable Analyst,https://cryptojobslist.com/jobs/accounts-payable-analyst-parity-technologies-london-england-united-kingdom,Tags Cryptocurrency Jobs Web3 Analyst Jobs Blockchain Operations Jobs Blockchain Finance Jobs Cryptocurrency Accounting Jobs Cryptocurrency ...,2023-02-17,London England United Kingdom
221,Binance Accelerator Programme Data analyst intern,https://cryptojobslist.com/jobs/binance-accelerator-programme-data-analyst-intern-binance-singapore,Tags Cryptocurrency Jobs Web3 Trading Jobs Web3 Internship Jobs Cryptocurrency Analyst Jobs Web3 Engineering Jobs Web3 Exchange Jobs Crypto...,2023-02-23,Singapore
1163,Technical Analyst,https://remoteOK.com/jobs/201886,1kx is an earlystage fund that helps founders launch and bootstrap token networks We primarily lead Seed and Series A investment rounds with a foc...,NaT,Remote Worldwide
1197,Validator Research Analyst,https://remoteOK.com/jobs/200374,As stewards of the Polkadot and Substrate ecosystem Parity is laying the foundation for a better web which respects the freedom and data of indivi...,NaT,Worldwide


## Sales - Worldwide

In [44]:
sales = worldwide[worldwide['title'].str.contains('sales', case=False) | worldwide['description'].str.contains('sales', case=False)]

In [45]:
sales.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_80120/3195154911.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  sales.describe()


,title,link,description,pubDate,location
count,8,8,8,5,8
unique,8,8,8,4,3
top,Aha Senior Product Marketing Manager,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Headquarters United States URL httpwwwahaio Aha helps companies build lovable software We provide the worlds 1 product development tools Aha Ro...,2023-03-01 00:00:00,NaN
freq,1,1,1,2,5
first,NaN,NaN,NaN,2023-02-16 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-03 00:00:00,NaN


In [46]:
sales

,title,link,description,pubDate,location
1353,Aha Senior Product Marketing Manager,https://weworkremotely.com/remote-jobs/aha-senior-product-marketing-manager,Headquarters United States URL httpwwwahaio Aha helps companies build lovable software We provide the worlds 1 product development tools Aha Ro...,2023-02-16,NaN
1324,GovAssist LLC Ad Operations Specialist,https://weworkremotely.com/remote-jobs/govassist-llc-ad-operations-specialist,Headquarters US URL httpsgovassistbamboohrcomjobs We are GovAssistcom GovAssist LLC is the leading company specialized in immigration consulta...,2023-02-28,NaN
1311,TimeZest Growth Marketing Manager,https://weworkremotely.com/remote-jobs/timezest-growth-marketing-manager,Headquarters Remote URL httpstimezestcom About TimeZest TimeZest is a B2B SaaS technology company that has reimagined how the IT Service indus...,2023-03-01,NaN
1309,Coachingcom Customer Success Agent,https://weworkremotely.com/remote-jobs/coaching-com-customer-success-agent,Headquarters United States URL httpscoachingcom GENERAL INFORMATIONDepartment Customer SuccessReporting to Support Manager Start Date Immediate...,2023-03-01,NaN
1116,Senior Product Designer Growth at Apollo,https://nodesk.co/remote-jobs/apollo-senior-product-designer-growth/,Apollo is hiring a remote Senior Product Designer Growth This is a fulltime position that can be done remotely anywhere in the United States Apoll...,2023-03-03,NaN
1162,DevOps Engineer,https://remoteOK.com/jobs/201896,At Figment our mission is to create an Internet that is truly decentralized where users can freely interact share collaborate and exchange goods a...,NaT,Worldwide
1187,Head of Quality Assurance,https://remoteOK.com/jobs/200396,Accelerating the Web3 transformation journeySettleMint is here to make blockchain application development easy for developers by providing the bes...,NaT,Remote job
1207,Developer Advocate,https://remoteOK.com/jobs/199859,About Swirlds LabsSwirlds Labs was established with the mission to accelerate the future built on Hedera the most used greenest enterprisegrade p...,NaT,Worldwide


## Python - Worldwide

In [47]:
python = worldwide[worldwide['title'].str.contains('python', case=False) | worldwide['description'].str.contains('python', case=False)]

In [48]:
python.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_80120/349435123.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  python.describe()


,title,link,description,pubDate,location
count,16,16,16,8,16
unique,16,16,16,6,4
top,Sr DevOps Engineer New Zealand Fiji GMT12,https://cryptojobslist.com/jobs/sr-devops-engineer-new-zealand-fiji-gmt-12-polygon-auckland,Tags Cryptocurrency Jobs Web3 Remote Jobs Web3 Devops Jobs Blockchain Security Jobs Web3 Developer Jobs Blockchain Engineering Jobs Blockch...,2023-02-28 00:00:00,NaN
freq,1,1,1,2,9
first,NaN,NaN,NaN,2023-02-17 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-03 00:00:00,NaN


In [49]:
python

,title,link,description,pubDate,location
316,Sr DevOps Engineer New Zealand Fiji GMT12,https://cryptojobslist.com/jobs/sr-devops-engineer-new-zealand-fiji-gmt-12-polygon-auckland,Tags Cryptocurrency Jobs Web3 Remote Jobs Web3 Devops Jobs Blockchain Security Jobs Web3 Developer Jobs Blockchain Engineering Jobs Blockch...,2023-02-17,Auckland
1349,YAGRO LTD Senior DevOps Engineer,https://weworkremotely.com/remote-jobs/yagro-ltd-senior-devops-engineer,Headquarters Cambridge URL httpswwwyagrocom The agricultural industry is undergoing a profound change Tightening regulatory requirements reduc...,2023-02-20,NaN
221,Binance Accelerator Programme Data analyst intern,https://cryptojobslist.com/jobs/binance-accelerator-programme-data-analyst-intern-binance-singapore,Tags Cryptocurrency Jobs Web3 Trading Jobs Web3 Internship Jobs Cryptocurrency Analyst Jobs Web3 Engineering Jobs Web3 Exchange Jobs Crypto...,2023-02-23,Singapore
1321,XM Software Engineer in Test Python Cyprus Greece or Remote,https://weworkremotely.com/remote-jobs/xm-software-engineer-in-test-python-cyprus-greece-or-remote-2,Headquarters Cyprus Greece or Remote URL httpsxmcom Reference Number SDEP1022 The Role We are looking for an experienced SET Software Engi...,2023-02-28,NaN
1316,QuickMail Remote QA Engineer USD 4050k 4 daysweek,https://weworkremotely.com/remote-jobs/quickmail-remote-qa-engineer-usd-40-50k-4-days-week,Headquarters Europe URL httpsquickmailio If you dont think you meet all the criteria below but still are interested in the job please apply Nob...,2023-02-28,NaN
1292,FileCloud Senior Software Engineer C,https://weworkremotely.com/remote-jobs/filecloud-senior-software-engineer-cpp,Headquarters Austin TX URL httpswwwfilecloudcom FileCloud is looking for a Senior Software Engineer C with a strong track record to join our dy...,2023-03-02,NaN
1289,Publitascom Operations Engineer SaaS Remote,https://weworkremotely.com/remote-jobs/publitas-com-operations-engineer-saas-remote-1,Headquarters Netherlands URL httpspublitascom Create customer value We believe that business growth starts with a fantastic product that peopl...,2023-03-03,NaN
1285,Ivy ML Engineer Intern,https://weworkremotely.com/remote-jobs/ivy-ml-engineer-intern-4,Headquarters London GB URL httpsletsunifyai At Ivy were on a mission to unify all Machine Learning ML frameworks making ML code cleaner more fl...,2023-03-03,NaN
905,Fulltime Senior Full Stack Developer at Baserow,https://www.fossjobs.net/job/11192/senior-full-stack-developer-at-baserow/,Location WorldwideURL httpsbaserowiojobsexperiencedfullstackdeveloperDescription Baserow is on a mission to help everyone build applications faste...,NaT,NaN
908,Fulltime Senior Software Engineer Abstract Wikipedia at Wikimedia Foundation,https://www.fossjobs.net/job/11188/senior-software-engineer-abstract-wikipedia-at-wikimedia-foundation/,Location WorldwideURL httpsgrnhseb639756b1usDescription Senior Software Engineer Abstract Wikipedia Summary The Wikimedia Foundation is looking fo...,NaT,NaN


## SQL - Worldwide

In [50]:
sql = worldwide[worldwide['title'].str.contains('sql', case=False) | worldwide['description'].str.contains('sql', case=False)]

In [51]:
sql.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_80120/1629035654.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  sql.describe()


,title,link,description,pubDate,location
count,12,12,12,7,12
unique,12,12,12,6,4
top,Sr DevOps Engineer New Zealand Fiji GMT12,https://cryptojobslist.com/jobs/sr-devops-engineer-new-zealand-fiji-gmt-12-polygon-auckland,Tags Cryptocurrency Jobs Web3 Remote Jobs Web3 Devops Jobs Blockchain Security Jobs Web3 Developer Jobs Blockchain Engineering Jobs Blockch...,2023-02-17 00:00:00,NaN
freq,1,1,1,2,8
first,NaN,NaN,NaN,2023-02-17 00:00:00,NaN
last,NaN,NaN,NaN,2023-03-03 00:00:00,NaN


In [52]:
sql

,title,link,description,pubDate,location
316,Sr DevOps Engineer New Zealand Fiji GMT12,https://cryptojobslist.com/jobs/sr-devops-engineer-new-zealand-fiji-gmt-12-polygon-auckland,Tags Cryptocurrency Jobs Web3 Remote Jobs Web3 Devops Jobs Blockchain Security Jobs Web3 Developer Jobs Blockchain Engineering Jobs Blockch...,2023-02-17,Auckland
1352,EngagedMD DevOps Engineer,https://weworkremotely.com/remote-jobs/engagedmd-devops-engineer,Headquarters Washington DC URL httpsengagedmdcom Youre an experienced AWS dev ops engineer who is excited to make your mark on our growing engin...,2023-02-17,NaN
1349,YAGRO LTD Senior DevOps Engineer,https://weworkremotely.com/remote-jobs/yagro-ltd-senior-devops-engineer,Headquarters Cambridge URL httpswwwyagrocom The agricultural industry is undergoing a profound change Tightening regulatory requirements reduc...,2023-02-20,NaN
221,Binance Accelerator Programme Data analyst intern,https://cryptojobslist.com/jobs/binance-accelerator-programme-data-analyst-intern-binance-singapore,Tags Cryptocurrency Jobs Web3 Trading Jobs Web3 Internship Jobs Cryptocurrency Analyst Jobs Web3 Engineering Jobs Web3 Exchange Jobs Crypto...,2023-02-23,Singapore
1329,Skedda Platform Engineer Europe,https://weworkremotely.com/remote-jobs/skedda-platform-engineer-europe,Headquarters Melbourne Austrailia URL httpsskeddacom FULLTIME 100 Remote Within Europe or Central European Time Zone CET Ideal start timeline...,2023-02-27,NaN
1323,Jacks Flight Club Senior Full Stack TypeScript Developer,https://weworkremotely.com/remote-jobs/jack-s-flight-club-senior-full-stack-typescript-developer,Headquarters Worldwide Fully Remote URL httpsjacksflightclubcom We are looking for a Senior Full Stack Software Engineer to join our developmen...,2023-02-28,NaN
1289,Publitascom Operations Engineer SaaS Remote,https://weworkremotely.com/remote-jobs/publitas-com-operations-engineer-saas-remote-1,Headquarters Netherlands URL httpspublitascom Create customer value We believe that business growth starts with a fantastic product that peopl...,2023-03-03,NaN
904,Fulltime DevOps Engineer at Baserow,https://www.fossjobs.net/job/11193/devops-engineer-at-baserow/,Location WorldwideURL httpsbaserowiojobsdevopsengineerDescription Baserow is on a mission to help everyone build applications faster cheaper and e...,NaT,NaN
905,Fulltime Senior Full Stack Developer at Baserow,https://www.fossjobs.net/job/11192/senior-full-stack-developer-at-baserow/,Location WorldwideURL httpsbaserowiojobsexperiencedfullstackdeveloperDescription Baserow is on a mission to help everyone build applications faste...,NaT,NaN
908,Fulltime Senior Software Engineer Abstract Wikipedia at Wikimedia Foundation,https://www.fossjobs.net/job/11188/senior-software-engineer-abstract-wikipedia-at-wikimedia-foundation/,Location WorldwideURL httpsgrnhseb639756b1usDescription Senior Software Engineer Abstract Wikipedia Summary The Wikimedia Foundation is looking fo...,NaT,NaN
